# Semo Processing

## Semo Units
This notebook is for exploring the SEM-O registered units

In [2]:
import sys
import os
sys.path.insert(0, os.path.abspath('../'))

from lib.semo.units import units, generators
display(units())
display(generators())
display(generators()['FuelType'].value_counts())
display(generators()['FuelKind'].value_counts())

,ResourceType,FuelType,FuelKind
ResourceName,,,
IEU_ROIEWIC,INTERCONNECTOR_ERROR_UNIT,None,None
IEU_NIMOYLE,INTERCONNECTOR_ERROR_UNIT,None,None
I_ROIEWIC,INTERCONNECTOR,None,None
I_NIMOYLE,INTERCONNECTOR,None,None
GU_400050,GENERATOR,WIND,RENEWABLES
...,...,...,...
AU_500134,ASSETLESS_TRADING_UNIT,None,None
AU_500135,ASSETLESS_TRADING_UNIT,None,None
AU_500138,ASSETLESS_TRADING_UNIT,None,None


,FuelType,FuelKind
ResourceName,,
GU_400050,WIND,RENEWABLES
GU_400030,WIND,RENEWABLES
GU_400041,WIND,RENEWABLES
GU_400020,WIND,RENEWABLES
GU_400070,WIND,RENEWABLES
...,...,...
GU_404550,BATTERY,RENEWABLES
GU_404630,WIND,RENEWABLES
GU_405080,SOLAR,RENEWABLES


WIND            235
MULTI_FUEL       22
BATTERY          21
HYDRO            19
DISTILLATE       16
SOLAR            14
GAS              10
COAL              5
PUMP_STORAGE      4
OIL               4
PEAT              3
BIOMASS           2
Wind              2
SYNC_COMP         1
Name: FuelType, dtype: int64

RENEWABLES    298
MIXED          22
COAL_GAS       22
Name: FuelKind, dtype: int64

## Fuel Mix

Process semo meter data to calculate fuel mix

In [2]:
import sys
import os
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import numpy as np
sys.path.insert(0, os.path.abspath('../')) 

from lib.common import constants as k
from lib.semo import units as semo_units

file = k.RAW_DATA_DIR / 'semo' / 'meter_roi.csv'
data = pd.read_csv(file, index_col=0, parse_dates=True)

In [34]:
df = data.copy()[['StartTime', 'ResourceName', 'ResourceType', 'MeteredMW']]
df = df.loc[df['ResourceType'] == 'GEN']
df['StartTime'] = pd.to_datetime(df['StartTime'])
df = pd.merge(df, semo_units.generators(), on='ResourceName', how='left')
df = df.drop(columns=['FuelType', 'ResourceName', 'ResourceType'])
df = df.copy().pivot_table(
    index='StartTime',
    columns='FuelKind',
    values='MeteredMW',
    aggfunc='sum'
)
df = df.reset_index().rename_axis(None, axis=1)
df

,StartTime,COAL_GAS,OTHER,RENEWABLES
0,2023-09-27 23:00:00,214.14,769.354,2390.759
1,2023-09-27 23:30:00,214.44,788.578,2343.949
2,2023-09-28 00:00:00,217.92,787.761,2484.937
3,2023-09-28 00:30:00,216.18,772.219,2482.300
4,2023-09-28 01:00:00,215.07,784.884,2481.016
...,...,...,...,...
4317,2023-12-26 21:30:00,230.82,250.006,1245.599
4318,2023-12-26 22:00:00,221.92,247.691,1170.283
4319,2023-12-26 22:30:00,159.63,254.427,1183.046
4320,2023-12-26 23:00:00,120.35,257.407,1225.235


## Price Data

No-cleaning is needed !

Format the price data
* Imbalance Settlement Price
* Net Imbalance Volume (MV)

In [3]:
file = k.RAW_DATA_DIR / 'semo' / 'price_all.csv'
data = pd.read_csv(file, index_col=0, parse_dates=True)

## CSO Data


In [23]:
import sys
import os
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import numpy as np
sys.path.insert(0, os.path.abspath('../')) 

from lib.common import constants as k
from lib.semo import units as semo_units

raw = pd.read_csv(k.RAW_DATA_DIR / 'cso_meter_generation.csv')

In [33]:
df = raw.drop(raw.loc[raw['Time Bands'] == 'All time periods'].index)
df[['StartTime', 'EndTime']] = df['Time Bands'].str.split(' < ', expand=True)
df['Day'] = pd.to_datetime(df['Day'])
df['StartTime'] = df['Day'] + pd.to_timedelta(df['StartTime'].astype(str) + ':00')

In [68]:
frame = df.copy()
frame = frame[['StartTime', 'Primary Fuel Output', 'VALUE']]
frame = frame.rename(columns={ 'Primary Fuel Output': 'FuelType', 'VALUE': 'Value' })
frame

,StartTime,FuelType,Value
0,2020-01-01 00:00:00,Battery Storage,NaN
1,2020-01-01 00:00:00,Biomass/Peat,100.0300
2,2020-01-01 00:00:00,Coal,0.0000
3,2020-01-01 00:00:00,Distillate,0.0000
4,2020-01-01 00:00:00,Gas,980.0540
...,...,...,...
960381,2023-10-31 23:30:00,Waste,34.9817
960382,2023-10-31 23:30:00,Wind,1168.7789
960383,2023-10-31 23:30:00,Other Non-Renewable,0.8240
960384,2023-10-31 23:30:00,Other Renewable,10.1980


In [69]:
import stringcase
import re

frame['Value'].isna().sum()
frame = frame.fillna(0)
frame['FuelType'] = frame['FuelType'].apply(lambda x: stringcase.pascalcase(re.sub('/', '_', re.sub(' ', '_', re.sub('-', '_', x)))))
frame['FuelType'] = frame['FuelType'].apply(lambda x: f'Fuel_{x}')
frame

,StartTime,FuelType,Value
0,2020-01-01 00:00:00,Fuel_Battery_Storage,0.0000
1,2020-01-01 00:00:00,Fuel_Biomass_Peat,100.0300
2,2020-01-01 00:00:00,Fuel_Coal,0.0000
3,2020-01-01 00:00:00,Fuel_Distillate,0.0000
4,2020-01-01 00:00:00,Fuel_Gas,980.0540
...,...,...,...
960381,2023-10-31 23:30:00,Fuel_Waste,34.9817
960382,2023-10-31 23:30:00,Fuel_Wind,1168.7789
960383,2023-10-31 23:30:00,Fuel_Other_Non_Renewable,0.8240
960384,2023-10-31 23:30:00,Fuel_Other_Renewable,10.1980


In [72]:
fuel_mix = frame.pivot_table(
            index='StartTime',
            columns='FuelType',
            values='Value',
            aggfunc='sum'
        ).reset_index().rename_axis(None, axis=1)
fuel_mix = fuel_mix.drop(columns='Fuel_Net_Generation')
fuel_mix.to_csv(k.PROCESSED_DATA_DIR / 'fuel_mix.csv')
fuel_mix

,StartTime,Fuel_Battery_Storage,Fuel_Biomass_Peat,Fuel_Coal,Fuel_Distillate,Fuel_Gas,Fuel_Oil,Fuel_Other_Non_Renewable,Fuel_Other_Renewable,Fuel_Pumped_Storage_Hydro,Fuel_Renewable_Hydro,Fuel_Solar,Fuel_Waste,Fuel_Wind
0,2020-01-01 00:00:00,0.000,100.03,0.00,0.0,980.0540,18.089,2.033,10.3704,3.67,81.9843,0.0,38.4435,197.9506
1,2020-01-01 00:30:00,0.000,99.84,0.00,0.0,996.7950,18.023,2.008,10.1744,0.00,80.9895,0.0,38.8133,199.0086
2,2020-01-01 01:00:00,0.000,100.00,0.00,0.0,968.9470,18.024,2.000,10.1507,0.00,81.7406,0.0,38.6498,206.3622
3,2020-01-01 01:30:00,0.000,102.12,0.00,0.0,925.7760,18.058,1.999,10.2010,0.00,80.4649,0.0,38.4020,210.7427
4,2020-01-01 02:00:00,0.000,113.65,0.00,0.0,848.9040,18.009,1.999,10.2551,0.00,80.0499,0.0,39.0340,226.7687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67195,2023-10-31 21:30:00,0.000,27.87,52.69,0.0,403.4862,53.360,0.810,10.3091,5.57,81.2020,0.0,38.4208,1285.0529
67196,2023-10-31 22:00:00,0.517,27.88,52.61,0.0,445.7907,36.010,0.826,10.3051,5.97,81.1524,0.0,38.5795,1253.4188
67197,2023-10-31 22:30:00,0.000,27.01,49.67,0.0,344.6262,1.640,0.825,10.2352,4.64,79.2374,0.0,37.8158,1191.5799
67198,2023-10-31 23:00:00,0.002,20.81,49.29,0.0,331.8480,0.000,0.820,10.2337,0.61,80.3865,0.0,35.1075,1212.8452


In [77]:
fuel_mix.sum()

/var/folders/k1/yrhh617d1f57fx87nzql3mmm0000gn/T/ipykernel_43120/2329911001.py:1: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  fuel_mix.sum()


Fuel_Battery_Storage         8.203460e+03
Fuel_Biomass_Peat            2.900420e+06
Fuel_Coal                    7.023803e+06
Fuel_Distillate              2.064285e+05
Fuel_Gas                     5.655781e+07
Fuel_Oil                     2.040426e+06
Fuel_Other_Non_Renewable     8.507423e+04
Fuel_Other_Renewable         6.844517e+05
Fuel_Pumped_Storage_Hydro    1.054340e+06
Fuel_Renewable_Hydro         3.070757e+06
Fuel_Solar                   4.258989e+05
Fuel_Waste                   2.235439e+06
Fuel_Wind                    4.267798e+07
dtype: float64